In [1]:
""" Get tidal current from AVISO FES 2014 model. And write to file according to the buoy data's lon/ lat/ time. 
"""
import numpy as np
import netCDF4 as nc
import matplotlib
import matplotlib.pyplot as plt
import datetime
import pyfes ### has to use newpycode
import os
import sys
import re
from oceans.ocfis import spdir2uv
path_nhchi = os.path.expanduser('~/Documents/nhchi_work_2022/py_nhchi/')
# adding to the system path
sys.path.insert(0, path_nhchi)
from convert_time import datetime_to_yearday

In [2]:
### Data info
### buoy data info
year = '2023'
buoy_id = '42022'
path_met_buoy = '/Users/chi/Documents/Data/NDBC/'
fn_met = 'met_ndbc'+buoy_id+'_y'+year+'.txt'
path_cur_buoy = '/Users/chi/Documents/Data/NDBC/'
fn_cur = 'current_ndbc'+buoy_id+'_y'+year+'.txt'
lat_buoy = 27.505
lon_buoy = -83.741

In [3]:
### Info of buoy's adcp data
### LOAD CURRENT VELOCITY DATA ###
### Read buoy current data
nline_header = 2 ### need to check lines of headers manually
with open(path_cur_buoy+fn_cur, 'r') as f:
    nrow = len(f.readlines())
    print('Total Number of lines in the current data file:', nrow)
### get the headers & number of items in the header
count = 0
print('The caveat is that the file does not have exact same number of items in the header as in the data...')
with open(path_cur_buoy+fn_cur, 'r') as f:
    for line in f:
        if count == nline_header:
            line_1 = re.sub(r"\s+",',',line) ### replace space(s) with ','
            if line_1[-1] == ',':
                line_1 = line_1[:-1]
            break
        count = count + 1
### read through the file
line_1_list = line_1.split(',')
ncol = len(line_1_list) # number of items in the first line of data
arrayCur = np.nan*np.ones( (nrow-nline_header,ncol) ) ### record values only
count = 0
with open(path_cur_buoy+fn_cur) as f:
    for line in f:
        count = count + 1
        line_1 = re.sub(r"\s+",',',line)
        if line_1[-1] == ',':
                line_1 = line_1[:-1]
        # print(line_1)
        if count > nline_header:
            temp = line_1.split(',')
            temp = [float(item) for item in temp]
            nitem = len(temp)
            arrayCur[count-nline_header-1,0:nitem] = np.array(temp)
        
print('Shape of arrayCur:',arrayCur.shape)
arrayCur[arrayCur == 999] = np.nan

### convert to datetime
dtime_cur_buoy = []
for i in range( arrayCur.shape[0] ):
    dtime_cur_buoy.append( datetime.datetime(int(arrayCur[i,0]),int(arrayCur[i,1]),int(arrayCur[i,2]),int(arrayCur[i,3]),int(arrayCur[i,4])) )
dtime_cur_buoy = np.array( dtime_cur_buoy )

Total Number of lines in the current data file: 6977
The caveat is that the file does not have exact same number of items in the header as in the data...
Shape of arrayCur: (6975, 125)


In [4]:
### Use pyfes tool box to build the tide model
path_aviso = os.path.expanduser('~/Documents/Data/AVISO/auxiliary/tide_model/fes2014/')
ConfigFileU = path_aviso+'eastward_velocity.ini'
ConfigFileV = path_aviso+'/northward_velocity.ini'
eastward_velocity = pyfes.Handler("ocean","memory",ConfigFileU)
northward_velocity = pyfes.Handler("ocean", "memory",ConfigFileV)

In [7]:
### get tidal currents by specifying time and locations
u_tide = np.nan*np.ones( dtime_cur_buoy.shape )
v_tide = u_tide.copy()
for i in range( len(dtime_cur_buoy) ):
    u_tide[i], _, _ = eastward_velocity.calculate(np.array([lon_buoy]),np.array([lat_buoy]),np.array([dtime_cur_buoy[i]]))
    v_tide[i], _, _ = northward_velocity.calculate(np.array([lon_buoy]),np.array([lat_buoy]),np.array([dtime_cur_buoy[i]]))

In [8]:
### write tidal current data to txt file
header = 'datetime, longitude(deg), latitude(degN), utide(cm/s), vtide(cm/s). Data derived from AVISO FES2014 model.'
arr_out = np.column_stack( (dtime_cur_buoy, lon_buoy*np.ones(dtime_cur_buoy.shape), lat_buoy*np.ones(dtime_cur_buoy.shape), u_tide, v_tide) )
### write to a file
fileID = open('timeseries_uv-tide_'+year+'-buoy'+buoy_id+'.txt','w')
np.savetxt(fileID,arr_out,fmt='%s',header=header)
fileID.close()